In [17]:
import numpy as np
import pandas as pd

In [1]:
from transformers import pipeline

In [2]:
# list of pipeline applications here: https://huggingface.co/docs/transformers/v4.31.0/en/task_summary
# list of models: https://huggingface.co/models
#classifier = pipeline('sentiment-analysis')
classifier = pipeline('sentiment-analysis', model='ProsusAI/finbert') # this model was developed on financial data, so ...
                                                                      # it doesn't work well with non-financial statements...
                                                                      # like "this is dumb."

C:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hiren\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
res = classifier("The sky is blue.")

print(res) # tells you sentiment of the sentence

[{'label': 'negative', 'score': 0.4016461968421936}]


In [7]:
res = classifier("The stock market went up.")

print(res) # tells you sentiment of the sentence

[{'label': 'neutral', 'score': 0.4263584315776825}]


### Tune the model for your dataset

In [12]:
# from this website: https://huggingface.co/docs/transformers/training
# NOTE: Need to do "conda install datasets" first
import tensorflow as tf
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset = dataset["train"]  # Just take the training split for now

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [15]:
print(dataset)
print(dataset['sentence'][100])

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 8551
})
If you eat more, you want correspondingly less.


In [20]:
from transformers import AutoTokenizer
#alternate model = ProsusAI/finbert
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data = tokenizer(dataset["sentence"], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(dataset["label"])  # Label is already an array of 0 and 1

In [21]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))  # No loss argument!

model.fit(tokenized_data, labels)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


268/268 [==============================] - 2600s 10s/step - loss: 0.5022
